<a href="https://colab.research.google.com/github/Avery1493/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/module4-Hyperparameter-Tuning/LS_DS_424_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [0]:
!pip install category-encoders

     |████████████████████████████████| 102kB 2.3MB/s 


In [0]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler

from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

ModuleNotFoundError: ignored

In [0]:
# Load Data
# TelecomCustomer
from google.colab import files
uploaded = files.upload()

Saving Telco_Customer_Churn.csv to Telco_Customer_Churn.csv


In [0]:
data = pd.read_csv("Telco_Customer_Churn.csv")

In [0]:
df = data.copy()
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [0]:
# Female = 0
df["gender"] = np.where(df["gender"] == "Female", 0, 1)
# No Partner = 0
df["Partner"] = np.where(df["Partner"] == "No", 0, 1)
# No Dependents = 0
df["Dependents"] = np.where(df["Dependents"] == "No", 0, 1)
# No Service = 0
df["PhoneService"] = np.where(df["PhoneService"] == "No", 0, 1)
# Not paperless = 0
df["PaperlessBilling"] = np.where(df["PaperlessBilling"] == "No", 0, 1)
# Churn No = 0
df["Churn"] = np.where(df["Churn"] == "No", 0, 1)

In [0]:
df.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
2416,8982-NHAVY,1,0,0,0,27,1,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,One year,1,Bank transfer (automatic),100.50,2673.45,0
3834,2325-ZUSFD,0,0,1,1,57,1,No,DSL,Yes,No,No,No,Yes,Yes,One year,1,Bank transfer (automatic),70.10,3913.3,1
6919,5136-GFPMB,1,0,0,0,54,1,Yes,Fiber optic,No,No,Yes,No,No,Yes,Month-to-month,0,Credit card (automatic),89.40,4869.5,0
4895,1394-SUIUH,0,1,1,0,44,1,No,Fiber optic,Yes,Yes,No,Yes,No,No,Month-to-month,1,Electronic check,85.15,3670.5,0
3114,0867-LDTTC,1,0,0,0,5,1,No,DSL,No,No,Yes,Yes,Yes,Yes,Month-to-month,0,Bank transfer (automatic),75.15,392.65,0


In [0]:
df["TotalCharges"][488]

' '

In [0]:
# Train Test Split
train, test = train_test_split(df, test_size=.33)
print(train.shape)
print(test.shape)

(4718, 21)
(2325, 21)


In [0]:
# Target and Features
# Binary Classification
target = "Churn"
features = df.drop(["customerID", "TotalCharges", target], axis=1).columns.to_list()

x_train = train[features]
y_train = train[target]

x_test = test[features]
y_test= test[target]

In [0]:
# Processeing Pipeline

pipe = make_pipeline(
    ce.OneHotEncoder(),
    MinMaxScaler()
)

X_train = pipe.fit_transform(x_train)
X_test= pipe.transform(x_test)
print("X train shape", X_train.shape)

X train shape (4718, 39)


In [0]:
pipe.steps

[('onehotencoder',
  OneHotEncoder(cols=['MultipleLines', 'InternetService', 'OnlineSecurity',
                      'OnlineBackup', 'DeviceProtection', 'TechSupport',
                      'StreamingTV', 'StreamingMovies', 'Contract',
                      'PaymentMethod'],
                drop_invariant=False, handle_missing='value',
                handle_unknown='value', return_df=True, use_cat_names=False,
                verbose=0)),
 ('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1)))]

In [0]:
# Net Model

# Model
model = Sequential([
                    Dense(39, input_dim=39, activation="relu"),
                    Dropout(.2),
                    Dense(1, activation="sigmoid")
                    
])

# Compile Model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

#Fit Model
model.fit(X_train,y_train, validation_split=0.25, epochs=5)

Epoch 1/5
111/111 [==============================] - 0s 3ms/step - loss: 0.4972 - accuracy: 0.7547 - val_loss: 0.4514 - val_accuracy: 0.7737
Epoch 2/5
111/111 [==============================] - 0s 2ms/step - loss: 0.4363 - accuracy: 0.7937 - val_loss: 0.4343 - val_accuracy: 0.7983
Epoch 3/5
111/111 [==============================] - 0s 2ms/step - loss: 0.4304 - accuracy: 0.7959 - val_loss: 0.4296 - val_accuracy: 0.8017
Epoch 4/5
111/111 [==============================] - 0s 2ms/step - loss: 0.4245 - accuracy: 0.7993 - val_loss: 0.4277 - val_accuracy: 0.8042
Epoch 5/5
111/111 [==============================] - 0s 2ms/step - loss: 0.4192 - accuracy: 0.7979 - val_loss: 0.4261 - val_accuracy: 0.7983


In [0]:
# Evaluate Model
model.evaluate(X_test,y_test)

73/73 [==============================] - 0s 1ms/step - loss: 0.4579 - accuracy: 0.7806


[0.45789310336112976, 0.7806451320648193]

## Hyperparameter Tunning with Grid Search

In [0]:
# Hyperparameters
# batch_size
# training epochs
# optimizer
### learning rate (if applicable to optimizer)
### momentum (if applicable to optimizer)
# activation functions
# network weight initialization
# dropout regularization
# number of neurons in the hidden layer

In [0]:
# Prepare of KerasClassifier
# tune batch size and epochs

def create_model():
    # create model
    model = Sequential()
    model.add(Dense(39, input_dim=39, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Pass instance of model
model = KerasClassifier(build_fn=create_model, verbose=0)

# Parameters in fit statement
param_grid = {'batch_size': [5, 10],
              'epochs': [5]}

# Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.8054262399673462 using {'batch_size': 5, 'epochs': 5}


In [0]:
from tensorflow.keras.optimizers import Adam
# tune learning rate and momentum

def create_model(learning_rate=.01):
    # create model
    model = Sequential()
    model.add(Dense(39, input_dim=39, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Pass instance of model
model = KerasClassifier(build_fn=create_model, verbose=0)

# Parameters in fit statement
param_grid = {'learning_rate': [.001, .01, .02]}

# Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train, epochs=5, batch_size=5)
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.8028827428817749 using {'learning_rate': 0.001}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?